
# FDR-Consistent Sharpe Ratio Cutoff

This notebook reproduces the FDR-based cutoff \(SR_c\) as a function of \(P(H_1)\)

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import root_scalar



## Parameters (edit as needed)


In [ ]:

# Core parameters (matching the exercise)
q = 0.1
SR0 = 0.0
SR1 = 0.5
sigma0 = 0.204
sigma1 = 0.341

# Sweep range for P(H1)
P1_vals = np.linspace(0.01, 0.25, 50)



## Solver (SciPy root on a safe bracket)

We solve $f(x)=0$ where
$$
f(x) \equiv \alpha(x) - K\,\pi(x),\quad K = \frac{q\,P(H_1)}{(1-q)\,P(H_0)}\,.
$$

To avoid numerical underflow at extreme tails, we bracket the root inside a finite, safe interval and expand slightly only if needed. If the solution gives a flat line, try tightening the bracket.


In [ ]:

def solve_sr_cutoff(P1, q=0.25, SR0=0.0, SR1=0.5, sigma0=0.5, sigma1=0.5,
                    bracket=(0.0, 3.0), max_expand=6):
    P0 = 1.0 - P1
    if not (0.0 < P1 < 1.0):
        raise ValueError("P1 must be in (0,1).")
    K = q * P1 / ((1.0 - q) * P0)

    def f(x):
        alpha = 1.0 - norm.cdf((x - SR0) / sigma0)
        power = 1.0 - norm.cdf((x - SR1) / sigma1)
        return alpha - K * power

    a, b = bracket
    fa, fb = f(a), f(b)
    k = 0
    while fa * fb > 0 and k < max_expand:
        a = max(-1.0, a - 1.0)   # mild expansion down
        b = b + 1.0              # mild expansion up
        fa, fb = f(a), f(b)
        k += 1

    sol = root_scalar(f, bracket=[a, b], method="brentq")
    x = sol.root
    alpha_star = 1.0 - norm.cdf((x - SR0) / sigma0)
    return float(x), float(alpha_star)



## Compute and plot \(SR_c\) vs \(P(H_1)\)


In [ ]:

rows = []
for P1 in P1_vals:
    SR_c, alpha_star = solve_sr_cutoff(P1, q=q, SR0=SR0, SR1=SR1, sigma0=sigma0, sigma1=sigma1)
    rows.append({"P[H1]": P1, "SR_c": SR_c, "alpha*": alpha_star})
df = pd.DataFrame(rows)

plt.figure(figsize=(6,4))
plt.plot(df["P[H1]"]*100, df["SR_c"], marker='o')
plt.xlabel("P[H1] (%)")
plt.ylabel("SR_c threshold (non-annualized)")
plt.title(f"SR_c vs P[H1] (q={int(q*100)}%)")
plt.grid(True)
plt.savefig('SR_c.png',dpi=300)

df.to_csv("appendix_5.csv")
df.head(10)